# Practical Application III: Comparing Classifiers

**Overview**: In this practical application, your goal is to compare the performance of the classifiers we encountered in this section, namely K Nearest Neighbor, Logistic Regression, Decision Trees, and Support Vector Machines.  We will utilize a dataset related to marketing bank products over the telephone.  



### Getting Started

Our dataset comes from the UCI Machine Learning repository [link](https://archive.ics.uci.edu/ml/datasets/bank+marketing).  The data is from a Portugese banking institution and is a collection of the results of multiple marketing campaigns.  We will make use of the article accompanying the dataset [here](CRISP-DM-BANK.pdf) for more information on the data and features.



### Problem 1: Understanding the Data

To gain a better understanding of the data, please read the information provided in the UCI link above, and examine the **Materials and Methods** section of the paper.  How many marketing campaigns does this data represent?

In [ ]:
#bank-marketing-ml-classifiers
#Overall, the dataset represents results from 17 campaigns done over a period of 2.5 years (May'2008 to Nov'2010) corresponding to a total of 79354 contacts

### Problem 2: Read in the Data

Use pandas to read in the dataset `bank-additional-full.csv` and assign to a meaningful variable name.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.metrics import auc, roc_curve, accuracy_score, ConfusionMatrixDisplay, confusion_matrix
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import OneHotEncoder,StandardScaler
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.pipeline import Pipeline
from sklearn.compose import make_column_transformer, ColumnTransformer, make_column_selector

In [ ]:
url = 'https://raw.githubusercontent.com/ashrivastav33/bank-marketing-ml-classifiers/refs/heads/main/data/bank-additional-full.csv'
bank_mkt_df = pd.read_csv(url, sep = ';')

In [ ]:
bank_mkt_df.head()

In [ ]:
#Validating data is loaded correctly in the bank_mkt_df dataframe
bank_mkt_df.info()

In [ ]:
##Exploring statistical details for bank_mkt_df dataframe
bank_mkt_df.describe()

### Problem 3: Understanding the Features


Examine the data description below, and determine if any of the features are missing values or need to be coerced to a different data type.


```
Input variables:
# bank client data:
1 - age (numeric)
2 - job : type of job (categorical: 'admin.','blue-collar','entrepreneur','housemaid','management','retired','self-employed','services','student','technician','unemployed','unknown')
3 - marital : marital status (categorical: 'divorced','married','single','unknown'; note: 'divorced' means divorced or widowed)
4 - education (categorical: 'basic.4y','basic.6y','basic.9y','high.school','illiterate','professional.course','university.degree','unknown')
5 - default: has credit in default? (categorical: 'no','yes','unknown')
6 - housing: has housing loan? (categorical: 'no','yes','unknown')
7 - loan: has personal loan? (categorical: 'no','yes','unknown')
# related with the last contact of the current campaign:
8 - contact: contact communication type (categorical: 'cellular','telephone')
9 - month: last contact month of year (categorical: 'jan', 'feb', 'mar', ..., 'nov', 'dec')
10 - day_of_week: last contact day of the week (categorical: 'mon','tue','wed','thu','fri')
11 - duration: last contact duration, in seconds (numeric). Important note: this attribute highly affects the output target (e.g., if duration=0 then y='no'). Yet, the duration is not known before a call is performed. Also, after the end of the call y is obviously known. Thus, this input should only be included for benchmark purposes and should be discarded if the intention is to have a realistic predictive model.
# other attributes:
12 - campaign: number of contacts performed during this campaign and for this client (numeric, includes last contact)
13 - pdays: number of days that passed by after the client was last contacted from a previous campaign (numeric; 999 means client was not previously contacted)
14 - previous: number of contacts performed before this campaign and for this client (numeric)
15 - poutcome: outcome of the previous marketing campaign (categorical: 'failure','nonexistent','success')
# social and economic context attributes
16 - emp.var.rate: employment variation rate - quarterly indicator (numeric)
17 - cons.price.idx: consumer price index - monthly indicator (numeric)
18 - cons.conf.idx: consumer confidence index - monthly indicator (numeric)
19 - euribor3m: euribor 3 month rate - daily indicator (numeric)
20 - nr.employed: number of employees - quarterly indicator (numeric)

Output variable (desired target):
21 - y - has the client subscribed a term deposit? (binary: 'yes','no')
```



In [ ]:
# Looking for missing values
print(bank_mkt_df.isnull().sum())

In [ ]:
# It looks like there is no null data but some of the categorical variables have 'unknown', for the
# purpose of modelling we'll keep these categories as 'unknown'

In [ ]:
y = np.where(df['y']=='yes',1,0)
X = bank_mkt_df.drop(['y','duration'], axis=1)

In [ ]:
# Let's identify counts of unknown for each of the columns
X[X[['job','marital','education','default','housing','loan']] == 'unknown'].count() / X.count()

In [ ]:
X.info()

### Problem 4: Understanding the Task

After examining the description and data, your goal now is to clearly state the *Business Objective* of the task.  State the objective below.

In [ ]:
# The business objective is to develop a predictive algorithm that identifies customers likely to
# accept the offer based on a number customer characteristics - this will allow to focus the
# limited resources of the organization / call center on customers most likely to accept the offer
# and have a benefit of reducing frustration (due to calls) from customers definitely won't

### Problem 5: Engineering Features

Now that you understand your business objective, we will build a basic model to get started.  Before we can do this, we must work to encode the data.  Using just the bank information features, prepare the features and target column for modeling with appropriate encoding and transformations.

In [ ]:
basic_transformer = ColumnTransformer([('scale', StandardScaler(),\
        make_column_selector(dtype_include=['int64', 'float64'])),\
        ('encode', OneHotEncoder(handle_unknown = 'ignore',drop='if_binary'),\
        make_column_selector(dtype_include=[object]))], sparse_threshold = 0)

In [ ]:
X_transformed = pd.DataFrame(data = basic_transformer.fit_transform(X.iloc[:,:7]), columns = basic_transformer.get_feature_names_out())

#X_transformed = pd.DataFrame(data = basic_transformer.fit_transform(X), columns = basic_transformer.get_feature_names_out())

X_transformed.head()

In [ ]:
df_for_corr = pd.concat([X_transformed,pd.DataFrame(y)], axis = 1)
df_for_corr.rename(columns = {0:'y'}, inplace = True)

In [ ]:
import plotly.graph_objects as go

fig = go.Figure()
fig.add_trace(
    go.Heatmap(
        x = df_for_corr.corr().columns,
        y = df_for_corr.corr().index,
        z = np.array(df_for_corr.corr()
        )
    )
)
fig.update_layout(title = 'Correlation Matrix')

In [ ]:
# Let's identify parameters with highest correlation with y
df_for_corr.corr().sort_values('y', ascending=False)['y'].head(5)

### Problem 6: Train/Test Split

With your data prepared, split it into a train and test set.

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_transformed, y, test_size = .2, random_state = 32)

### Problem 7: A Baseline Model

Before we build our first model, we want to establish a baseline.  What is the baseline performance that our classifier should aim to beat?

In [ ]:
# Baseline accuracy is equal to the frequency of the most common class (0)
1-y.sum()/len(y)

### Problem 8: A Simple Model

Use Logistic Regression to build a basic model on your data.  

In [ ]:
import time
start_time = time.time()
lr = LogisticRegression(max_iter = 1000).fit(X_train,y_train)
lr_elapsed_time = time.time() - start_time
print(f'{lr_elapsed_time:.2f}s elapsed during training')

### Problem 9: Score the Model

What is the accuracy of your model?

In [ ]:
acc_sc = accuracy_score(y_test, lr.predict(X_test))
print(acc_sc)

### Problem 10: Model Comparisons

Now, we aim to compare the performance of the Logistic Regression model to our KNN algorithm, Decision Tree, and SVM models.  Using the default settings for each of the models, fit and score each.  Also, be sure to compare the fit time of each of the models.  Present your findings in a `DataFrame` similar to that below:

| Model | Train Time | Train Accuracy | Test Accuracy |
| ----- | ---------- | -------------  | -----------   |
|     |    |.     |.     |

In [ ]:
start_time=time.time()
knn = KNeighborsClassifier().fit(X_train,y_train)
knn_elapsed_time = time.time() - start_time
print(f'{knn_elapsed_time:.2f}s elapsed during training')

In [ ]:
start_time=time.time()
dt = DecisionTreeClassifier().fit(X_train,y_train)
dt_elapsed_time = time.time() - start_time
print(f'{dt_elapsed_time:.2f}s elapsed during training')

In [ ]:
start_time=time.time()
svm = SVC().fit(X_train,y_train)
svm_elapsed_time = time.time() - start_time
print(f'{svm_elapsed_time:.2f}s elapsed during training')

In [ ]:
results = pd.DataFrame({'model':['Logistic Regression',' KNN', 'Decision Tree', 'SVM'],\
                       'Train Time':[lr_elapsed_time,knn_elapsed_time,dt_elapsed_time,svm_elapsed_time],\
                       'Train Accuracy': [accuracy_score(y_train, lr.predict(X_train)), accuracy_score(y_train,knn.predict(X_train)),\
                        accuracy_score(y_train, dt.predict(X_train)),accuracy_score(y_train, svm.predict(X_train))],\
                       'Test Accuracy':[accuracy_score(y_test, lr.predict(X_test)), accuracy_score(y_test,knn.predict(X_test)),\
                        accuracy_score(y_test, dt.predict(X_test)),accuracy_score(y_test, svm.predict(X_test))]})

In [ ]:
results.set_index('model')

### Problem 11: Improving the Model

Now that we have some basic models on the board, we want to try to improve these.  Below, we list a few things to explore in this pursuit.


- Hyperparameter tuning and grid search.  All of our models have additional hyperparameters to tune and explore.  For example the number of neighbors in KNN or the maximum depth of a Decision Tree.  
- Adjust your performance metric

In [ ]:
lr_params = {'penalty': ['l1', 'l2'], 'C': np.linspace(0.01,100, 1000)}

In [ ]:
lr_grid = RandomizedSearchCV(lr, param_distributions = lr_params, scoring = 'roc_auc', n_jobs = -1)

In [ ]:
start_time=time.time()
lr_grid.fit(X_t_train,y_train)
lrg_elapsed_time = time.time() - start_time
print(f'{lrg_elapsed_time:.2f}s elapsed during training')

In [ ]:
knn_params = {'n_neighbors': range(1,len(y_test),2)}

In [ ]:
knn_grid = RandomizedSearchCV(knn, param_distributions = knn_params, scoring = 'roc_auc', n_jobs = -1)

In [ ]:
start_time=time.time()
knn_grid.fit(X_t_train,y_train)
kng_elapsed_time = time.time() - start_time
print(f'{kng_elapsed_time:.2f}s elapsed during training')

In [ ]:
dt_params = {'criterion': ['gini','entropy','log_loss'], 'max_depth':range(1,15,1), 'ccp_alpha' : range(0,1,100)

In [ ]:
dt_grid = RandomizedSearchCV(dt, param_distributions = dt_params, scoring = 'roc_auc', n_jobs = -1)

In [ ]:
start_time=time.time()
dt_grid.fit(X_t_train,y_train)
dtg_elapsed_time = time.time() - start_time
print(f'{kng_elapsed_time:.2f}s elapsed during training')

In [ ]:
grid_summary = pd.DataFrame({'model':['Logistic Regression','KNN','Decision Tree'],\
        'run time': [lrg_elapsed_time, kng_elapsed_time, dtg_elapsed_time],\
        'auc score': [lr_grid.best_score_, knn_grid.best_score_, dt_grid.best_score_]})

In [ ]:
grid_summary.set_index('model')

In [ ]:
lrfpr, lrtpr, lrthresholds = roc_curve(y_test, lr_grid.best_estimator_.predict_proba(X_t_test)[:,1])
knnfpr, knntpr, knnthresholds = roc_curve(y_test, knn_grid.best_estimator_.predict_proba(X_t_test)[:,1])
dtfpr, dttpr, dtthresholds = roc_curve(y_test, dt_grid.best_estimator_.predict(X_t_test))

In [ ]:
fig = px.area(
    x=knnfpr, y=knntpr,
    title=f'ROC Curve for KNN (best AUC) classifier',
    labels=dict(x='False Positive Rate', y='True Positive Rate'),
    width=700, height=500
)
fig.add_shape(
    type='line', line=dict(dash='dash'),
    x0=0, x1=1, y0=0, y1=1
)

fig.update_yaxes(scaleanchor="x", scaleratio=1)
fig.update_xaxes(constrain='domain')
fig.show()


In [ ]:
#Let's say we can only cover 50% of the population. Given the imbalanced nature of classes we'll ned to
#adjust the threshold

optimal_idx = np.argmax(knntpr - knnfpr)
optimal_threshold = knnthresholds[optimal_idx]
print("Threshold value is:", optimal_threshold)

In [ ]:
# new predictions w/ adjusted threshold
knn_y_test_adj = np.where(knn_grid.best_estimator_.predict_proba(X_t_test)[:,1]>knnthresholds[optimal_idx],\
                          1,0)

In [ ]:
# Using the adjusted threshold, we get the following confusion matrix
cm = confusion_matrix(y_test, knn_y_test_adj)
disp = ConfusionMatrixDisplay(confusion_matrix=cm)
disp.plot()
plt.show()